In [18]:
import os
from math import *
from time import perf_counter_ns
import gc
from random import choice
import copy
os.chdir('dataset')

# Parse Input

In [3]:
def parse_input_file(file_address):
    # store vertices and edges in separate sets
    vertices = set()
    edges = [] 
    # read file
    with open(file_address) as f:
        lines = f.readlines()
    num_V_E = [int(x) for x in lines[0].split()]
    lines = lines[1:]
    # populate sets
    for line in lines:
        x = line.split()
        vertices.add(x[0])
        vertices.add(x[1])
        edges.append((x[0], x[1], int(x[2])))
    vertices = list(vertices)
    return vertices, edges, num_V_E

In [54]:
graphs = []
for file in os.listdir():
    vertices, edges, num_V_E = parse_input_file(file) 
    graphs.append((num_V_E, vertices, edges))

In [6]:
#graphs #a list of graphs as tuples, the tuples contain: 
#[0]: a list containing number of vertices and edges
#[1]: a list of vertices' names
#[2]: a list of edges as tuples (u,v,w)

# Stoer and Wagner's deterministic algorithm 

In [67]:

def findCut(V,E):
        rV,rW, nV = findMax(V,E)
        if nV != None:
            E,_,_ = updateDict(rV,nV,E)
        return rW, E

def findMax(V,E):
    Ecop = copy.deepcopy(E)
    seen = []
    V2 = None
    while(len(Ecop) > 2):
        highestWeight = Ecop[V][0]
        Ecop, V, V2 = updateDict(V,highestWeight[0],Ecop)
    remainingVertex = list(Ecop.keys())
    remainingWeight = Ecop[remainingVertex[0]][0][1]
    V1 = remainingVertex[0]
    return V1, remainingWeight, V2

def trueLast(listOfV):
    x = listOfV[1].rfind('&')
    return listOfV[0],listOfV[1][x+1:]

def updateDict(oldTouched,newTouched, EdgeDict, doSort = True):
    list1 = EdgeDict[oldTouched]
    list2 = EdgeDict[newTouched]
    newName = oldTouched+'&'+newTouched
    EdgeDict.pop(oldTouched)
    EdgeDict.pop(newTouched)
    EdgeDict[newName] = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if list1[i] == [] or list2[j] == []:
                continue
            if list1[i][0] == oldTouched or list1[i][0] == newTouched:
                list1[i] = []
                continue
            if list2[j][0] == oldTouched or list2[j][0] == newTouched:
                list2[j] = []
                continue
            if list1[i][0] == list2[j][0]:
                EdgeDict[newName].append((list1[i][0],list1[i][1]+list2[j][1]))
                list1[i] = []
                list2[j] = []
    
    for i in range(len(list1)):
        if list1[i] == []:
            continue
        EdgeDict[newName].append((list1[i][0],list1[i][1]))
        list1[i] = []
    for j in range(len(list2)):
        if list2[j] == []:
            continue
        EdgeDict[newName].append((list2[j][0],list2[j][1]))
        list2[j] = []
        
    for u,w in EdgeDict[newName]:
        if u == oldTouched:
            u = newName
        if u == newTouched:
            u = newName
        listOld = EdgeDict[u]
        for i in range(len(listOld)):
            if listOld[i][0] == oldTouched or listOld[i][0] == newTouched:
                listOld[i] = (newName,w)
        listOld = list(dict.fromkeys(listOld))
        if doSort == True:
            EdgeDict[u] = sorted(listOld,reverse=True, key=lambda e:e[1])
        else:
            EdgeDict[u] = listOld
        EdgeDict = deleteSelfRef(EdgeDict)
    return EdgeDict, newName, newTouched

def deleteSelfRef(EdgeDict):
    for selfName in list(EdgeDict.keys()):
        listSelf = EdgeDict[selfName]
        for i in range(len(listSelf)):
            if listSelf[i][0] == selfName:
                listSelf.pop(i)
                break
    return EdgeDict

def createDict(verticesTuple): #this creates a dictionary with "v" as key and (u,w) as values, also sorts it! for faster access.
    vert = {}
    for (u,v,w) in verticesTuple:
        if u not in vert:
            vert[u] = []
        if v not in vert:
            vert[v] = []
        vert[u].append((v,int(w)))
        vert[v].append((u,int(w)))
    for v in vert:
        vert[v] = sorted(vert[v],reverse=True, key=lambda e:e[1])
    return vert

In [68]:
#start with arbitary vertex
graph1 = graphs[0]
leftVertices = graph1[0][0]
vertices = graph1[1]
edges = createDict(graph1[2])
startV = vertices[0]
cuts=[]
while leftVertices > 2:
    cutWeight, edges = findCut(startV,edges)
    cuts.append(cutWeight)
    leftVertices -= 1
print(min(cuts))
#while there is more than one vertex
##find a cut (find a node it's edge to V is maximized, then create a supernode, find a vertex
##that has the highest weight to the super node, go on until we reach the last two! make these a super node!)
##make the graph smaller!(continue what we did last time and store the last weight each time, minimum of them is the min cut)

3056


# Karger and Stein's randomized algorithm

In [78]:
#contraction of an edge:
#while len V > 2
# pick edge from E randomly
# contract E
#return only cut left!
graph1 = graphs[0]
edges = createDict(graph1[2])
cuts=[]
for i in range(0,10):
    leftVertices = graph1[0][0]
    Ecop = copy.deepcopy(edges)
    while leftVertices > 2:
        randV = choice(list(Ecop.keys()))
        randE = choice(Ecop[randV])
        Ecop,_,_ = updateDict(randV,randE[0],Ecop,False)
        leftVertices -=1
    cuts.append(Ecop[list(Ecop.keys())[0]][0][1]) #a very ugly way to get the weight but it works!
print(min(cuts))


3056
